## Data Formatting

In [7]:
import pandas as pd

import json
from time import sleep
import openai

DEFAULT_SYSTEM_PROMPT = 'You are customer support bot. You should help to user to answer on his question.'

def get_example(question, answer):
    return {
        "messages": [
            {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
            {"role": "user", "content": question},
            {"role": "assistant", "content": answer},
        ]
    }

if __name__ == "__main__":
    df = pd.read_csv("dataset.csv")
    with open("train.jsonl", "w") as f:
        for i, row in list(df.iterrows()):
            question = row["question"]
            answer = row["answer"]
            example = get_example(question, answer)
            example_str = json.dumps(example)
            f.write(example_str + "\n")

## Train your model

In [4]:
!pip install openai

In [8]:
 openai.api_key = "<YOUR API KEY HERE>"

In [ ]:
def wait_untill_done(job_id):
    events = {}
    while True:
        response = openai.FineTuningJob.list_events(id=job_id, limit=10)
        # collect all events
        for event in response["data"]:
            if "data" in event and event["data"]:
                events[event["data"]["step"]] = event["data"]["train_loss"]
        messages = [it["message"] for it in response.data]
        for m in messages:
            if m.startswith("New fine-tuned model created: "):
                return m.split("created: ")[1], events
        sleep(10)

if __name__ == "__main__":
    response = openai.File.create(file=open("train.jsonl", "rb"), purpose="fine-tune")
    uploaded_id = response.id
    print("Dataset is uploaded")
    print("Sleep 30 seconds...")
    sleep(30)  # wait until dataset would be prepared
    response = openai.FineTuningJob.create(training_file=uploaded_id,model="gpt-3.5-turbo")
    print("Fine-tune job is started")
    ft_job_id = response.id
    new_model_name, events = wait_untill_done(ft_job_id)
    with open("new_model_name.txt", "w") as fp:
        fp.write(new_model_name)
    print(new_model_name)

## Use your fine-tunned Model

In [ ]:
DEFAULT_SYSTEM_PROMPT ='You are customer support bot. You should help to user to answer on his question.'

def get_fine_tuned_model_name():
    with open("new_model_name.txt") as fp:
        return fp.read()

def call_openai(model_name, messages):
    response=openai.ChatCompletion.create(
        messages=messages,
        model=model_name
    )
    return response.choices[0]["message"]["content"]

if __name__=="__main__":
    model_name = get_fine_tuned_model_name()
    history = [
            {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
        ]
    while True:
        user_input = input("User: ")
        if user_input.lower() == 'exit':
            print("AI: Goodbye!")
            break
        history.append({'role': 'user', 'content': user_input})
        response = call_openai(model_name, history)
        print("AI:", response)
        history.append({'role': :'assistant', 'content': response})